# End to End Pure Streaming Data-Pipeline for Contractor Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read contractor state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "contractor_streamingDF" --> "results_df" --> "contractor_data"

In [0]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [0]:
# Schema for Contractor
contractor_schema = StructType([
            StructField("Contract_id", IntegerType(), False),
            StructField("Name", StringType(), True),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True)])

contractor_udf_schema = StructType([
            StructField("Name", StringType(), True),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get contractor csv files as a streaming "contractor_streamingDF" and process it on the fly and get transformed stream "contractor_df"
###### Objective: (incoming csv files) --> "contractor_streamingDF" --> "contractor_df"

In [0]:
# Get contractor Steaming DataFrame from csv files

# streaming starts here by reading the input files 
contractor_Path = "/FileStore/apartment/contractor/inprogress/"
contractor_streamingDF = (
  spark
    .readStream
    .schema(contractor_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(contractor_Path)
)
# Clear invalid rows
contractor_df = contractor_streamingDF.select("*").where("Contract_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(contractor_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['Name', 'Address_line_1', 'City', 'Post_code', 'Region'])

Out[ 5 ]: <optimus.df_transformer.DataFrameTransformer at 0x7f2e7f752cc0>

##### This function parses the corresponding columns into a single column

In [0]:
def my_fun(Name, Address_line_1, City, Post_code, Region, event_time):
  return zip(Name, Address_line_1, City, Post_code, Region, event_time)

udf_Fun = udf(my_fun, ArrayType(contractor_udf_schema))

In [0]:
intermediate_df = ( contractor_df.withWatermark("event_time", "10 seconds")
            .groupBy("Contract_id")
            .agg(F.collect_list("Name").alias("Name"),
                 F.collect_list("Address_line_1").alias("Address_line_1"),
                 F.collect_list("City").alias("City"),
                 F.collect_list("Post_code").alias("Post_code"),
                 F.collect_list("Region").alias("Region"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Contract_id", 
                    F.explode(udf_Fun(F.column("Name"), 
                                      F.column("Address_line_1"), 
                                      F.column("City"), 
                                      F.column("Post_code"),
                                      F.column("Region"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [0]:
results_df = (intermediate_df
              .select("Contract_id", 
                      "data.Name", 
                      "data.Address_line_1", 
                      "data.City", 
                      "data.Post_code", 
                      "data.Region",
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Contract_id")

##### Display final result
###### This result shows the latest state of all the unique contractor_id

In [0]:
display(results_df)

Contract_id,Name,Address_line_1,City,Post_code,Region,event_time,latest_event_time
1,Brunhilde Norway,3 Bonner Crossing,Reston,22096,Virginia,2017-06-01T10:59:12.000+0000,2017-06-01T10:59:12.000+0000
2,Clement Stellino,1653 Westport Court,Baton Rouge,70894,Louisiana,2017-10-16T07:07:17.000+0000,2017-10-16T07:07:17.000+0000
3,Jasen Binford,12 Rusk Park,Port Washington,11054,New York,2017-07-05T04:57:03.000+0000,2017-07-05T04:57:03.000+0000
4,Quinn Rymell,7 Barnett Way,Gaithersburg,20883,Maryland,2017-11-09T22:40:51.000+0000,2017-11-09T22:40:51.000+0000
5,Roderick Gullam,4 Springs Center,Pittsburgh,15225,Pennsylvania,2017-02-15T14:53:05.000+0000,2017-02-15T14:53:05.000+0000
6,Garey Garret,1 Granby Way,Anniston,36205,Alabama,2018-06-27T05:55:13.000+0000,2018-06-27T05:55:13.000+0000
7,Benedick Lavallin,02 Forest Trail,Charleston,25331,West Virginia,2017-07-22T16:40:58.000+0000,2017-07-22T16:40:58.000+0000
8,Brigg Sandbatch,29 Bowman Circle,Raleigh,27610,North Carolina,2018-08-31T14:49:12.000+0000,2018-08-31T14:49:12.000+0000
9,Lewie Degli Antoni,198 Kingsford Place,Vero Beach,32964,Florida,2017-08-18T12:27:07.000+0000,2017-08-18T12:27:07.000+0000
10,Micky Weightman,25305 Helena Terrace,New York City,10034,New York,2016-12-05T22:20:13.000+0000,2016-12-05T22:20:13.000+0000


##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [0]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/contractor/_checkpoints/streaming-agg")
 .start("/delta/apartment/contractor_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe will always be Up-To-Date

In [0]:
contractor_data = spark.read.format("delta").load("/delta/apartment/contractor_data").orderBy("Contract_id")

In [0]:
display(contractor_data)

Contract_id,Name,Address_line_1,City,Post_code,Region,event_time,latest_event_time
1,Brunhilde Norway,3 Bonner Crossing,Reston,22096,Virginia,2017-06-01T10:59:12.000+0000,2017-06-01T10:59:12.000+0000
2,Clement Stellino,1653 Westport Court,Baton Rouge,70894,Louisiana,2017-10-16T07:07:17.000+0000,2017-10-16T07:07:17.000+0000
3,Jasen Binford,12 Rusk Park,Port Washington,11054,New York,2017-07-05T04:57:03.000+0000,2017-07-05T04:57:03.000+0000
4,Quinn Rymell,7 Barnett Way,Gaithersburg,20883,Maryland,2017-11-09T22:40:51.000+0000,2017-11-09T22:40:51.000+0000
5,Roderick Gullam,4 Springs Center,Pittsburgh,15225,Pennsylvania,2017-02-15T14:53:05.000+0000,2017-02-15T14:53:05.000+0000
6,Garey Garret,1 Granby Way,Anniston,36205,Alabama,2018-06-27T05:55:13.000+0000,2018-06-27T05:55:13.000+0000
7,Benedick Lavallin,02 Forest Trail,Charleston,25331,West Virginia,2017-07-22T16:40:58.000+0000,2017-07-22T16:40:58.000+0000
8,Brigg Sandbatch,29 Bowman Circle,Raleigh,27610,North Carolina,2018-08-31T14:49:12.000+0000,2018-08-31T14:49:12.000+0000
9,Lewie Degli Antoni,198 Kingsford Place,Vero Beach,32964,Florida,2017-08-18T12:27:07.000+0000,2017-08-18T12:27:07.000+0000
10,Micky Weightman,25305 Helena Terrace,New York City,10034,New York,2016-12-05T22:20:13.000+0000,2016-12-05T22:20:13.000+0000


### Do Some Live Streaming Graphs

In [0]:
contractor_data_stream = spark.readStream.format("delta").load("/delta/apartment/contractor_data")

In [0]:
display(contractor_data_stream.groupBy("Region").count())

Region,count
Minnesota,1
Ohio,3
Texas,2
Pennsylvania,4
Nebraska,2
Nevada,1
Illinois,1
District of Columbia,2
Alaska,1
West Virginia,2
